# SQL in Python - Connecting to and retrieving data from PostgreSQL
Previously, you have learned how to connect to a SQL database by using a SQL client such as DBeaver.  
Apart from connecting to databases, DBeaver also allows you to run SQL queries against the database, create new tables and populate them with data as well as retrieving the data.  
Populating tables with data that you have locally on your machine usually requires you to save it in a file, like a CSV, and import it using the DBeaver UI.  
Often, before you reached the final step of uploading your dataset, you have performed data cleaning procedures to bring your data into shape.  
This is usually done in Python (or R). This means we would import the data into Python, clean it, export it to a CSV file, import it into DBeaver and upload the data into the database.  
This process requires multiple steps and more than one software. Fortunately, we can reduce the steps by connecting to the database from Python directly, eliminating the need for a separate SQL client. 
  
**This notebook will not only teach you how to connect to a database and query its data in Python directly, but also how to automate the process using custom functions and modules.**

## Creating a connection to a PostgreSQL database with Python
The go-to package in Python for connecting to a PostgreSQL database is called <ins>psycopg2</ins>. Check out their official documentation here: https://www.psycopg.org/docs/.  
Complete the code below to import the package.

In [ ]:
# Import psycopg2 package


In order to create a connection to our PostgreSQL database we need the following information:
- host = the address of the machine the database is hosted on
- port = the virtual gate number through which communication will be allowed
- database = the name of the database
- user = the name of the user
- password = the password of the user

This information was provided to you prior to the "Introduction to Databases" lecture .  
The function from the psycopg2 package to create a connection to the database is called <ins>connect()</ins>. The connect() function expects the parameters listed above as input in order to connect to the database.  
  
We created a separate file called sql.md to store the required connection parameters. What's important is that the file is listed in .gitignore which prevents it from being pushed to the remote repository. Now we don't have to worry about the credentials becoming exposed online.  

Let's import the parameters host, port, database, user and password from the sql.py file. First, rename sql.md to sql.py, then open the sql.py file, enter your credentials and run the code block below. This will import the parameters from the external file into this notebook. Don't worry if you don't know or understand this way of importing yet, we will explain it later in the notebook.

In [ ]:
# Import connection parameters: host, port, database, user, password
from sql import host, port, database, user, password

We have imported the variables from the sql.py file and they're now available in this notebook. Print the user variable to check if it worked.

In [ ]:
# Print user variable


Next, we're going to pass the imported variables that hold the credential information to the beforementioned connect() function and create a connection object called conn.  
Complete the code below and assign the imported variables to the right parameter of the connect() function.

In [ ]:
# Create connection object conn
conn = psycopg2.connect(host=____,
                        port=____,
                        database=_______,
                        user=_____,
                        password=________)

Next, let's have a look at the conn variable to see what we're working with.

In [ ]:
# Print connection object conn


The conn variable is a connection object or more precisely, a database session. This means that currently we have one open connection to our database.  
This connection will stay open until we manually close it. 

## Retrieving data from the database
Before we can use our connection to get data, we have to create a cursor. A cursor allows Python code to execute PostgreSQL commmands in a database session.  
A cursor has to be created with the <ins>cursor()</ins> method of our connection object conn.

In [ ]:
# Create cursor: cur
cur = conn._______

# Print cursor


Now, in order to retrieve data from our database, we have to use our cursor and its <ins>execute()</ins> method. The execute() method takes a SQL query as parameter and executes it.  
Complete the code below to select all columns and the top 5 rows from the flights table.

In [ ]:
# Use execute() and print top 5 rows from flights 
cur.execute('SELECT ___ FROM ___ LIMIT ___')

# Print cursor


When printing the cursor, we still don't get any data output.  
Why? Because our cursor behaves similar to our connection. The result in stored inside the cursor object and can't be accessed by simply printing it.  
Instead we need to use the <ins>fetchall()</ins> method, which fetches all rows of a query result.  
Complete the code below and print the results of the SQL query.

In [ ]:
# Use fetchall()
cur.________

There we go! Finally we have the output of our SQL query.  
Now, before we summarise everything we have done, let's close our database connection to free up database resources for our colleagues.

In order to close a database connection we have to use the <ins>close()</ins> method of our connection ebject.
Complete the code below and terminate the database connection.

In [ ]:
# Close database connection
conn.________

Perfect, let's summarise the steps we have performed above:
1. (Install and) Import the psycopg2 package
2. Create a database connection object with the connect() method
3. Create a cursor for the database connection with the cursor() method
4. Use the execute() method of the cursor to execute a SQL query
5. Use the fetchall() method of the cursor to retrieve the results of the SQL query
6. Close the database connection with the close() method

In total we needed about 6 steps to connect to the databse and retrieve data.  
Even though we probably won't have to go through all the steps over and over again when querying data, this is still somewhat of a tedious process. On top of that, there is another inconvenience when it comes to the output. The format of the data we retrieve from the database is a list. As analysts we almost always want to have our data in a dataframe, since it makes exploring and cleaning data a lot easier.

Let's fix these problems and do the following:
1. Write a custom function that performs all of the steps above
2. Change the code so the SQL output is in a dataframe

## Automating data retrieval with a custom function: get_data()
Instead of having to write multiple lines of code whenever we want to query data from Python, we're going to make our lives easier by writing a custom function that will execute all of the necessary steps automatically. For this, we're going to define a custom function called <ins>get_data()</ins> below. This function should only expect one argument: query. The function should be able to take any query as a string, create a connection to the database, execute the query, output the data and close the connection.

Complete the code below so that the get_data() function prints the output of any SQL query we pass it.

In [ ]:
def get_data(_____):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # create a connection to the PostgreSQL server
        conn = ___________________
		
        # create a cursor
        cur = __________
        
	    # execute the sql query
        cur.____________

        # display the results of the query
        cur.____________
       
	    # close the connection to the PostgreSQL database
        _________

    # the code below makes the function more robust, you can ignore this part
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

Now it's time to check if your function works. Use the get_data() function below to return the top five rows of the flights table.

In [ ]:
# Print top 5 rows from flights
get_data('____________')

It works, awesome! Now whenever we want to connect to our database and retrieve data we can simply use the get_data() function, how convenient! Unforturnately, we can't call ourselves Python hackers yet, because we still need to solve the problem with the output format.

## Changing the SQL query output format from list to dataframe
We know already, that the output format of the fetchall() function is a list, which is inconvenient to work with. Luckily there is a function that let's us read a SQL query directly into a dataframe. It's called <ins>read_sql_query()</ins> and can be found in the pandas package.
The function expects a SQL query as the first argument and a connection (object) to a database as the second argument.
Complete the code below and return the output in a dataframe.

In [ ]:
# Import pandas package
import pandas as pd

# Print top 5 rows from flights table using read_sql_query()
__.read_sql_query('_________________', ____)

This output looks like a dataframe. Was it really that easy? Let's check if the output really is a dataframe.  
Complete the code below and check if the output is of type dataframe.

In [ ]:
# Print type of read_sql_query() output


It really is a dataframe! Wow, that was easy and what's even better: we can don't need a cursor and also don't need to run the execute() and fetchall() functions anymore!  
The only thing left is to change our get_data() function and replace the steps that are not needed anymore with our new read_sql_query function.  
Copy the content of the code block where we define the get_data() function and paste it into the code block below. Then, adjust the code by replacing redundant steps with our new read_sql_query() function.

In [ ]:
# Copy get_data() and implement read_sql_query()


Let's make sure the function works by returning the first 5 rows of the flights table below.

In [ ]:
# Print top 5 rows from flights


Now that we have the data in a dataframe, it's easy to apply all the different data exploration and cleaning techniques we have learned already. We don't have to do that right now, but it will become very useful in the future!  
Congratulations, from now on you can call yourself a Python hacker!  

But wait..., what if I told you there is a way of making your get_data() function even more powerful?  

## Automating data retrieval using a Python module
Before I will tell how to make your function even more powerful, let's take a step back and look at the prerequisites for using the get_data() function and what limitations apply.  
In order to use our get_data() function anywhere in our jupyter notebook, we need to define it once with the correct connection details.  
But what if we have multiple jupyter notebooks in our repository and we want to use this in other notebooks as well? How could we do that?  
As with all things in life: <ins>it depends</ins>.  

Remember that when you start a jupyter notebook, you simultaneously start a kernel that will then load your virtual environment and execute any Python code you feed it.  
Every package you load and every function you define will be available in that jupyter notebook only.  
Now imagine you have defined the get_data() function in only one of your jupyter notebooks, but you have multiple notebooks in your repository where you would like to use this function.
To be able to do that you would have to add the function definition to each notebook. In reality you would simply duplicate your code over and over, which should always be avoided. While this wouldn't mean the end of the world, the real problem occurs if we ever wanted to change the connection details inside the function. In that case, we would have to find and change each function definition in each file, which not only increases the likelihood of errors but could become a tedious unnecessarily time consuming task.  

Now that we know about the downsides, can you think of a another, better solution? If you don't have any idea, think about Python packages or more specifically about why and how we use them.  
The answer is, we import Python packages because they include predefined functions that we can then use in any of our Python scripts, which is exactly what we want to do with our function. You're probably thinking: "But we don't have a package and we also don't want to create a package for just function and publish it to the offical Python library". That's true but fortunately, we don't have to. Instead we can use a built-in Python functionality called <ins>modules</ins>.    
Instead of giving you a detailed explanation what modules are and what they do, let's create a python module and use it instead.

Let's get started!

Please perform the following steps:
1. Open sql.py and import all Python packages necessary for the get_data() function to work
2. Insert the definition of the get_data() function
3. Import the Python file into this jupyter notebook below the same way you normally import packages and give it the alias gd
4. Execute the get_data() function and retrieve the top 5 rows from flights

**Important: To make sure we're using the function from our get_data.py module and not the one we defined earlier in this jupyter notebook we need to give the imported package an alias and use it to reference the get_data() function**

In [ ]:
# Import get_data() from sql.py
import sql as gd

# Print top 5 rows from flights
gd.________('SELECT * FROM flights LIMIT 5')

Worked like a charm!  
Great, let's summarise: Whenever we want to use a custom function in any other jupyter notebook in our repository, we  

1. create a new Python module,
2. add the necessary packages and function definitions,
3. and import it into our jupyter notebook using the standard import snytax

If you still don't understand what modules are and why we did what we did, check out the offical [documentation](https://docs.python.org/3/tutorial/modules.html) about modules or reach out to us. It's important you understand this concepts since we will be working with python modules in future jupyter notebooks.

Congratulations for making it through this notebook, you deserve to call yourself a badass Python hacker!